<a href="https://colab.research.google.com/github/scskalicky/creative_naming_lsa/blob/main/creative_names_cogsci.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# manually download the most up-to-date English model (lg)
# 
# MUST RESTART RUNTIME AFTER DOING THIS.
# will also update spacy to 3.x
#!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.2.0/en_core_web_lg-3.2.0.tar.gz

In [1]:
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_lg')

In [4]:
a = 'air ball'
b = 'air toy'

nlp(a).similarity(nlp(b))

0.747491200736556

In [ ]:
dat = pd.read_csv('/content/drive/MyDrive/creative_naming/creative_naming_raw_answers.csv')

In [ ]:
dat

,subject,group,item,original_answer,cleaned_answer,on.task
0,49998,NES,balloon,air ball /,air ball,1
1,49998,NES,box,Compressed paper container,compressed paper container,1
2,49998,NES,carrot,orange veggie,orange veggie,1
3,49998,NES,fork,eating tool,eating tool,1
4,49998,NES,shovel,digger,digger,1
...,...,...,...,...,...,...
1291,R_Zres5zjqgT36pEt,ROM,fork,comb for food,comb for food,1
1292,R_Zres5zjqgT36pEt,ROM,glasses,clarifier,clarifier,1
1293,R_Zres5zjqgT36pEt,ROM,shovel,plant remover,plant remover,1
1294,R_Zres5zjqgT36pEt,ROM,toilet,pee-pee taker,pee pee taker,1


The goal of this notebook is to see whether or not we can use wordnet to determine the relative heirarchy of categories in the participants answers.


What are the common linguistic properties of high frequency answers?

- repeated answers
- similar answers
- unique answers

What does this mean for creativity? 


So we need a way to group similar answers based on

- the words used
  - ideally we would want to categorize things
- their semantic similarity to one another
- their syntax
- orthographic overlap? 


In [23]:
from nltk.corpus import wordnet as wn
from nltk.corpus import framenet as fn
from operator import itemgetter

## FrameNet

- Each LU is linked to a Frame, and hence to the other words which evoke that Frame.

Get all the frames for target object? Then see how the answers relate to those frames? Hrmm. 

In [37]:
fn.frames_by_lemma(r'toilet')

[<frame ID=78 name=Containers>]

In [40]:
fn.frames_by_lemma(r'wallet')

[<frame ID=78 name=Containers>]

In [44]:
fn.frames_by_lemma(r'glasses')

[<frame ID=158 name=Accoutrements>]

In [58]:
fn.frames_by_lemma(r'orange')

[]

In [67]:
def find_frames(answer):
  answer_tokens = nltk.word_tokenize(answer)
  for token in answer_tokens:
    print(token)
    print(fn.frames_by_lemma(token))

In [68]:
find_frames('orange veggie')

orange
[<frame ID=423 name=Color>]
veggie
[]


In [69]:
def find_wordnet(answer):
  answer_tokens = nltk.word_tokenize(answer)
  for token in answer_tokens:
    print(token)
    print(wn.synsets(token))

Answers like "comb for food" are creating metaphors or unique blends - attribution of one function to another.

Answers like "digger" are nouning the verb fuction of the object

Is the answer a subordinate or superordinate entity when compared to the given entity? Silly question since we provided items that will all be relatively specific? 



In [73]:
find_wordnet('digger')

digger
[Synset('digger.n.01'), Synset('power_shovel.n.01')]


In [101]:
# set the prompt object as a wordnet variable
balloon = wn.synset('balloon.n.02')
box = wn.synset('box.n.01')
carrot = wn.synset('carrot.n.01')
couch = wn.synset('couch.n.01')
fork = wn.synset('fork.n.01')
sunglasses = wn.synset('sunglasses.n.01')
shovel = wn.synset('shovel.n.01')
wallet = wn.synset('wallet.n.01')
toilet = wn.synset('toilet.n.01')

prompt_objects = [balloon, box, carrot, couch, fork, sunglasses, shovel, wallet, toilet]

In [111]:
for po in prompt_objects:
  # the lexical representation
  print(po.lemma_names())
  # the definition
  print(po.definition())
  # hypernyms
  print(f'Hypernyms: {po.hypernyms()}')
  print(f'Hyponyms: {po.hyponyms()}')
  print('-----------')
  # the other synsets using that same lexical representation
 # print(po.lemmas())

['balloon']
small thin inflatable rubber bag with narrow neck
Hypernyms: [Synset('plaything.n.01')]
Hyponyms: []
-----------
['box']
a (usually rectangular) container; may have a lid
Hypernyms: [Synset('container.n.01')]
Hyponyms: [Synset('ballot_box.n.01'), Synset('bandbox.n.01'), Synset('carton.n.02'), Synset('casket.n.02'), Synset('cereal_box.n.01'), Synset('chest.n.02'), Synset('cigar_box.n.01'), Synset('coffin.n.01'), Synset('crate.n.01'), Synset('flat.n.02'), Synset('hod.n.01'), Synset('mailbox.n.01'), Synset('matchbox.n.01'), Synset('paintbox.n.01'), Synset('pencil_box.n.01'), Synset('playbox.n.01'), Synset('poor_box.n.01'), Synset('pyxis.n.03'), Synset('shadow_box.n.01'), Synset('shoebox.n.02'), Synset('snuffbox.n.01'), Synset('strongbox.n.01'), Synset('tinderbox.n.02'), Synset('tuck_box.n.01'), Synset('window_box.n.01')]
-----------
['carrot']
deep orange edible root of the cultivated carrot plant
Hypernyms: [Synset('root.n.01')]
Hyponyms: []
-----------
['sofa', 'couch', 'lou

In [94]:
[token.tag_ for token in nlp('pee pee taker')]

['NNP', 'NNP', 'NN']